In [1]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 18.5 MB/s eta 0:00:00


#Detecção de faixa de preservação de rios

##Introdução
De acordo com a legislação ambiental brasileira, os corpos hídricos devem ser acompanhados por Áreas de Preservação Permanente (APPs) ao longo de suas margens. A largura dessas faixas de preservação varia conforme a largura do próprio rio, com o objetivo de proteger a vegetação nativa, evitar processos erosivos e preservar a qualidade da água.

Este projeto tem como objetivo aplicar técnicas de visão computacional em imagens de satélite para detectar rios e analisar automaticamente se suas margens estão respeitando os limites exigidos por lei para as APPs.

#Desenvolvimento

Existem satelites cujas imagens são acessiveis gratuitamente, como o Sentinel-2, sendo necessário apenas registrar-se. O Sentinel-2 também possui datasets com imagens que são compostas por varias imagens unidas, de forma a remover nuvens e sombras. Cada dataset desse é composto por um arquivo .tif contedo a imagem de uma área para cada banda do espectro registrada, como a Azul, Verde, Vermelha e InfraVermelho.

[Aqui é possível encontrar imagens mosaicos do Sentinel-2](https://browser.stac.dataspace.copernicus.eu/collections/sentinel-2-global-mosaics)

#**Definindo o caminho para os arquivos de imagem (importante para conseguir rodar)**
Como os arquivos da imagem de satelite são muito grandes (cerca de 600 MB no total), a melhor forma de utilizá-los no Google Colab sem precisar baixá-los a cada execução é salvar no Google Drive e acessá-los montando o Drive à instância do colab.

Instruções para utilizar o código como está:


1.   [Link para o Drive com os arquivos utilizados](https://drive.google.com/drive/folders/1UvuaiLn4GmCVSSatLX-ALwVZoWUEUH8O?usp=sharing)
2.   clicar no triangulo ao lado de "Compartilhados comigo > visao computacional"
3.   ir em "Organizar", "Adicionar Atalho", "Todos os Locais" e adicionar em "Meu Drive"
4.   Depois, no Colab, na aba lateral com os arquivos da instância, habilitar a opção "Mount Drive"

In [2]:
green_band_path = '/content/drive/MyDrive/visao computacional/sentinel 2 mosaico interior sp/B03.tif'
nir_band_path = '/content/drive/MyDrive/visao computacional/sentinel 2 mosaico interior sp/B08.tif'
red_band_path = '/content/drive/MyDrive/visao computacional/sentinel 2 mosaico interior sp/B04.tif'
blue_band_path = '/content/drive/MyDrive/visao computacional/sentinel 2 mosaico interior sp/B02.tif'

O arquivo .tif possui a resolução do satelite. Esse valor indica o tamanho da área representada por cada pixel da imagem. Nesse caso, cada pixel representa uma área de 10m x 10m, consistente com a resolução de 10m do Sentinel-2.

In [3]:
import rasterio

with rasterio.open('/content/drive/MyDrive/visao computacional/sentinel 2 mosaico interior sp/B02.tif') as src:
    print("Resolução (pixel size):", src.res)

Resolução (pixel size): (10.0, 10.0)


Essa resolução não é a melhor para analisar rios estreitos, pois um rio com menos de 10 metros de largura pode nem aparecer na imagem. Nesse caso, existem satelites comerciais, como o PlanetScope, que possuem resoluções maiores. No entanto, ainda podemos analisar rios maiores e sua faixa de APP com as imagens do Sentinel-2.

# Juntar bandas para visualização
Começamos por ler cada banda, e juntar as RGB para poder visualizar a imagem do satelite

In [4]:
import numpy as np
import cv2

with rasterio.open(blue_band_path) as blue_src:
    blue_band = blue_src.read(1, masked=True)  # le os valores da banda presente no arquivo
    blue_band = blue_band.filled(blue_band.min())  # Há valores invalidos, preenche eles com um valor placeholder
with rasterio.open(green_band_path) as green_src:
    green_band = green_src.read(1, masked=True)
    green_band = green_band.filled(green_band.min())
with rasterio.open(red_band_path) as red_src:
    red_band = red_src.read(1, masked=True)
    red_band = red_band.filled(red_band.min())
with rasterio.open(nir_band_path) as nir_src:
    nir_band = nir_src.read(1, masked=True)
    nir_band = nir_band.filled(nir_band.max())

In [5]:
# para criar a imagem RGB, combina as 3 bandas. Para isso, é necessário tranformar elas de int16 para 8bit, e normalizar os valores de 0 a 255
combined_image = cv2.merge([np.uint8(cv2.normalize(blue_band, None, 0, 255, cv2.NORM_MINMAX)),
                            np.uint8(cv2.normalize(green_band, None, 0, 255, cv2.NORM_MINMAX)),
                            np.uint8(cv2.normalize(red_band, None, 0, 255, cv2.NORM_MINMAX))])

# Por algum motivo, a imagem fica meio escura
# Aumenta o brilho da imagem
combined_image = cv2.convertScaleAbs(combined_image, alpha=3, beta=0)
#Salva imagem para visualização
cv2.imwrite('combined_image.png', combined_image)

True

# Segmentar rios
## Metodo NDWI
NDWI é um metodo para detectar bacias hidrograficas em imagens de satélites.

$\text{NDWI} = \frac{\text{Verde} - \text{NIR}}{\text{Verde} + \text{NIR}}$

Valores mais proximos de 1 indicam agua. Podemos utilizar o NDWI como uma nova banda da imagem, realçando rios e reservatorios de água.

In [6]:
# calculo NDWI com um numero minusculo adicionado ao denominador para evitar divisão por 0
ndwi = (green_band - nir_band) / (green_band + nir_band + 1e-10)
print(ndwi.min(), ndwi.max())
# Valores invalidos nas bandas podem gerar um NDWI fora do padrão (deve ser entre -1 e 1)
# Clipa valores que estão fora do padrão
ndwi = np.clip(ndwi, -1, 1)

# Salva NDWI para um png, para visualização, para isso, normaliza os valores entre 0 e 255 (adiciona 1 para modificar o intervalo de -1,1 para 0,2 e multiplica por 127.5 (que é 255/2) para chegar de 0,255 )
cv2.imwrite('ndwi_output.png', ((ndwi + 1) * 127.5).astype(np.uint8))

-111.0000000111 1000000000000.0


True

### Geração mascara binaria

Geramos uma mascara binaria onde apenas pixels com valor de NDWI acima de 0 recebem o valor 1.

In [7]:
#NDWI maior que um threshold (nesse caso 0) pode ser considerado agua (o valor de threshold pode ser modificado)
water_mask = np.where(ndwi > 0, 1, 0)

#salva mascara para visualização
cv2.imwrite('water_mask.png', (water_mask*255).astype(np.uint8))

True

Podemos tratar essa mascara para tentar diminuir os ruidos

In [8]:
from skimage.morphology import remove_small_objects, closing, disk
from scipy.ndimage import binary_fill_holes

#remover objetos menores que um numero dado (5)
water_mask = remove_small_objects(water_mask.astype(bool), min_size=5)

#conectar objetos separados por muito pouco (tenta unir segmentos de rios separados)
water_mask = closing(water_mask, disk(3))

#preenche buracos em objetos
water_mask = binary_fill_holes(water_mask)

cv2.imwrite('water_mask_tratada.png', (water_mask*255).astype(np.uint8))

True

# Calculo da largura do rio

## Utilizando Distance Transform e Skeletonization
O distance transform é uma técnica usada para calcular a distância de cada pixel em uma imagem até o pixel mais próximo de um objeto (normalmente representado por pixels de valor 1 em uma máscara binária). Quando aplicado em uma máscara binária de rios, ele transforma a imagem de forma que cada pixel da água recebe o valor da distância até a borda da água. O medial axis (ou eixo medial) pode então ser calculado a partir dessa transformação, representando a linha central dos rios. A largura do rio em cada ponto é obtida medindo a distância entre o eixo medial e as bordas do rio, aproveitando os valores do distance transform.

In [9]:
from skimage.morphology import medial_axis

# função medial_axis já retorna o eixo medial e o valor da distancia em cada ponto desse eixo
skeleton, distance_on_skeleton = medial_axis(water_mask, return_distance=True)

###Visualizando o esqueleto

In [10]:
# visualização do esqueleto
cv2.imwrite('esqueleto.png', (skeleton * 255).astype(np.uint8))

True

In [11]:
#Para sobrepor o esqueleto com a imagem de satelite, precisa criar um overlay igual a imagem
overlay = combined_image.copy()
#fazer assim certifica que vai usar NumPy (é possível acabar usando estruturas nativas do python e demorar anos pra rodar)
skeleton_indices = np.where(skeleton)

#os indices são os locais dos pixels do esqueleto, então pinta esses pixels de branco no overlay
overlay[skeleton_indices] = [255, 255, 255]   # Branco

#transpoem o overlay na imagem (por isso é bom o overlay ser igual a imagem, se usar uma matriz com zeros, a imagem escurece)
blend = cv2.addWeighted(combined_image, 0, overlay, 1, 0)

cv2.imwrite('satelite_com_esqueleto.png', blend)

True

# Geração da APP apropriada

In [12]:
#função para calcular a APP
def app(largura):
  if largura == 1:
    return 3
  elif largura > 1 and largura <= 5:
    return 5
  elif largura > 5 and largura <= 20:
    return 10
  elif largura > 20 and largura <= 60:
    return 20
  elif largura > 60:
    return 50

Para a criação de uma máscara com a APP apropriada, podemos, para cada pixel do esqueleto, gerar círculos com raio igual à soma da distância até a margem e da largura da APP apropriada.

In [13]:
# mascara para APPs
app_mask = np.zeros_like(water_mask, dtype=np.uint8)

# Iterar pelos pixels na mascara com o esqueleto
for y in range(skeleton.shape[0]):
    for x in range(skeleton.shape[1]):
        if skeleton[y, x]:  # se o pixel faz parte do esqueleto
            distance = distance_on_skeleton[y, x] #pega o valor da distancia da margem pra esse pixel

            # cria um circulo de raio (distancia até a margem + largura da app apropriada)
            # a largura do rio seria a distancia do centro até a margem * 2
            radius = int(distance + app(distance*2))
            if radius > 0:
                cv2.circle(app_mask, (x, y), radius, 1, thickness=-1) #thickness=-1 faz com que seja um circulo e não só uma circunferencia

#visualização
cv2.imwrite('app_mask_circulos.png', (app_mask * 255).astype(np.uint8))


True

A máscara gerada também comtem áreas que fazem parte do rio. Para remover essas áreas e garantir que apenas a APP seja representada, comparamos a máscara criada com a water_mask, que indica as regiões ocupadas pela água. Quando ambas as máscaras têm valor 1 na mesma posição, removemos esse pixel da máscara da APP.

In [14]:
# AND entre a mascara com as APPS e o inverso da water_mask
app_mask = app_mask & (1 - water_mask)

#visualização
cv2.imwrite('app_mask.png', (app_mask * 255).astype(np.uint8))


True

In [15]:
#igual a visualização do esqueleto
overlay = combined_image.copy()
mask_indices = np.where(app_mask)

overlay[mask_indices] = [255, 255, 255]   # Branco

blended = cv2.addWeighted(combined_image, 0.7, overlay, 0.3, 0)

# Save the resulting overlay
cv2.imwrite('satelite_com_app.png', blended)


True

# Detecção de transgressões em APPs

### Utilizando NDVI para detectar desmatamento

NDVI é um indice utilizado para medir a saúde da vegetação. Vegetação mais densa tem valores proximos de 1, enquanto areas sem nenhuma vegetação tem valores proximos do -1.

In [16]:
#calculo ndvi
ndvi = (nir_band - red_band) / (nir_band + red_band + 1e-10)
#certificar que ndvi está corretamente entre -1 e 1
print(ndwi.min(), ndwi.max())
#visualização, igual ao de ndwi
cv2.imwrite('ndvi_output.png', ((ndvi + 1) * 127.5).astype(np.uint8))

-1.0 1.0


<ipython-input-16-2a013865a338>:6: RuntimeWarning: invalid value encountered in cast
  cv2.imwrite('ndvi_output.png', ((ndvi + 1) * 127.5).astype(np.uint8))


True

###Mascara de desmatamento
Assim como no caso da mascara com NDWI, geramos uma mascara de desmatamento estabelecendo um limiar para o NDVI.

In [17]:
defos_mask = np.where(ndvi < 0.4, 1, 0)
cv2.imwrite('defos_mask.png', (defos_mask*255).astype(np.uint8))

True

Geramos uma nova mascara, contendo as areas de APPs desmatadas, fazendo a intersecção da mascara de APPs com a de desmatamento.

In [18]:
#dilata um pouquinho os rios para remover eles da mascara de desmatamento
#porque a margem imediata dos rios não tem vegetação e estava sendo marcada como desmatamento
defos_mask = defos_mask & (1 - cv2.dilate(water_mask.astype(np.uint8), np.ones((3, 3), np.uint8), iterations=2))
cv2.imwrite('desmatamento_mask.png', (defos_mask*255).astype(np.uint8))
#as areas de APPs desmatadas seriam a intersecção da mascara de app com a de desmatamento
app_transgression = app_mask & defos_mask
cv2.imwrite('transgressoes_app_mask.png', (app_transgression*255).astype(np.uint8))

True

#Visualizaçao Final
Combinando a mascara de APPs demarcadas, a mascara de transgressões em APPs, e a imagem de satelite, podemos visualizar o resultado

In [19]:
# Mesmo procedimento de sobreposição na imagem de satelite, mas com dois overlays
app_overlay = combined_image.copy()
mask_indices_app = np.where(app_mask)

app_transgression_overlay = combined_image.copy()
mask_indices_app_transgression = np.where(app_transgression)

app_overlay[mask_indices_app] = [255, 255, 255]  # Branco para as APPs
app_transgression_overlay[mask_indices_app_transgression] = [0, 0, 255]    # Vermelho para as transgressões

alpha_restricted = 0.3  # 30% de transparencia para o branco
alpha_transgression = 0.5  # 50% de transparencia para o vermelho

# sobrepoem os dois à imagem de satelite
blended = cv2.addWeighted(combined_image, 1 - alpha_restricted, app_overlay, alpha_restricted, 0)
blended = cv2.addWeighted(blended, 1 - alpha_transgression, app_transgression_overlay, alpha_transgression, 0)

# salva para visualização
cv2.imwrite('satellite_with_two_overlays.png', blended)

True

Como a mascara binaria é uma matriz com 1 ou 0, podemos calcular a porcentagem de transgressões somando todos os 1s de cada mascara

In [20]:
print(app_transgression.sum()/app_mask.sum())

0.14299636854138842
